In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticle
from msmrd2.integrators import integratorMAPK 

In [ ]:
# Define arbitrarily large list of patchy particles
np.random.seed(seed=1) # seed 1 good used for pentameric ring formation 
numparticles = 3
boxsize = 3
D = 1.0
Drot = 1.0
pyPartlist = [] 

for i in range(numparticles):    
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 2.0:
                overlap = True
                continue
                
    orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    orientation = orientation/np.linalg.norm(orientation)
    part = msmrd2.particle(D, Drot, position, orientation)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 2 # seed 2 good used for pentameric ring formation; seed = -1 uses random device as seed
bodytype = 'rigidbody' # three rotational degrees of freedom
anglePatches = np.pi
reactivationRateK = 0.5
reactivationRateP = 0.5
mapkIndex = [0]
kinaseIndex = [1]
phosIndex = [2]
integrator = integratorMAPK(dt, seed, bodytype, anglePatches, reactivationRateK, reactivationRateP, 
                            mapkIndex, kinaseIndex, phosIndex) 
integrator.setKbT(0.1)

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define MAPK potntial based on patchy particles
sigma = 1.0
strength = 60 #65
# This values are fixed and should match those used to determine metastable states in potential and trajectory.
patchesCoordinates1 = [np.array([np.cos(anglePatches/2), np.sin(anglePatches/2), 0.]), 
                       np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.])]
patchesCoordinates2 = [np.array([1.,0.,0.])]
# Defines patchy protein potential
potentialPatchyProteinMS = patchyProteinMarkovSwitch(sigma, strength, angularStrength, 
                                                     patchesCoordinates1, patchesCoordinates2)
# Incorporate potential into integrator
integrator.setPairPotential(potentialPatchyProteinMS)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 5000000
stride = 2500 #1000
datafile  = open('../../data/vmd/patchyParticles.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(3*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
            v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticles(numparticles = numparticles, 
                                     outfname = "patchyParticles", 
                                     tclfname = "../../data/vmd/patchyParticles_2vmd.tcl")
print("Percentage complete: ", 100, " %")